In [57]:
import pandas as pd
from transformers import AutoTokenizer
from transformers import TFAutoModelForSequenceClassification
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
# from tensorflow.keras.optimizers import Adam
# from tensorflow.keras.losses import SparseCategoricalCrossentropy
# from tensorflow.keras.models import load_model
# import pickle
import tensorflow as tf
# import tensorflow_hub as hub
# import keras
# from keras import layers
# from tensorflow.keras.layers import Input, Dense
# from sklearn.model_selection import train_test_split

# Load df, X & y

In [25]:
#df = pd.read_csv('/workspaces/spam/df.csv')
df = pd.read_csv('/Users/antoinebertin/Documents/jedha/full_stack/projects_full_stack/spam/df.csv')

# Load Model

In [4]:
loaded_model = load_model('16D_20epochs.keras')  # Load the model

In [5]:
loaded_model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, 59, 16)         │       320,016 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling1d        │ (None, 16)             │             0 │
│ (GlobalAveragePooling1D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 16)             │           272 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            17 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 960,917 (3.67 MB)

 Trainable params: 320,305 (1.22 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 640,612 (2.44 MB)

# Load Custom Tokenizer

In [6]:
# To load the tokenizer next time
with open('/workspaces/spam/tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)

# Split

In [26]:
df.drop(columns=['v1', 'v2', 'v2_encoded'], inplace=True)

In [3]:
# Assuming 'df' is your DataFrame
texts = df['v2_clean'].values
labels = df['encoding'].values
# X = tf.keras.preprocessing.sequence.pad_sequences(df['v2_encoded'], padding="post")
# y = df['encoding']

In [37]:
x_train, x_val, y_train, y_val = train_test_split(texts, labels, test_size=0.3, stratify=labels)

In [ ]:
# train = tf.data.Dataset.from_tensor_slices((x_train, y_train))
# val = tf.data.Dataset.from_tensor_slices((x_val, y_val))

In [ ]:
# train_batch = train.shuffle(len(train)).batch(64)
# val_batch = val.shuffle(len(val)).batch(64)

In [31]:
df.head()

,encoding,v2_clean
0,0,jurong point crazy available bugis great world...
1,0,ok lar joke wif oni
2,1,free entry wkly comp win fa cup final tkts 21s...
3,0,dun early hor
4,0,nah think usf live


## Transfer Learning

In [32]:
tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-cased")

In [46]:
# Convert your numpy array of texts to a list of strings
texts_list = texts.tolist()
split_texts = [str(sentence) for sentence in texts_list]
#flat_list = [item for sublist in split_texts for item in sublist]

In [51]:
len(split_texts)

5572

In [8]:
# len(flat_list)

41092

In [48]:
tokenized_data = tokenizer(split_texts, return_tensors="np", padding=True)

In [49]:
# Tokenizer returns a BatchEncoding, but we convert that to a dict for Keras
tokenized_data = dict(tokenized_data)

In [52]:
type(labels) # labels = np.array(dataset["label"])

numpy.ndarray

In [53]:
# Load and compile our model
#model = TFAutoModelForSequenceClassification.from_pretrained("google-bert/bert-base-cased")
model = TFAutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=2)

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [54]:
model.compile(optimizer='adam')

In [55]:
model.fit(tokenized_data, labels)

Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
175/175 [==============================] - 880s 5s/step - loss: 0.4456


In [62]:
#model.save('bert.keras')

In [58]:
# Visualization of the training process on the loss function
plt.plot(model.history["loss"], color="b", label="Training loss")
plt.plot(model.history["val_loss"], color="r", label="Validation loss")
plt.ylabel("Loss")
plt.xlabel("Epochs")
plt.legend(loc="best")
plt.show()

TypeError: 'History' object is not subscriptable

# Test

In [67]:
#'jurong point crazy available bugis great world la buffet cine amore wat' 0

In [70]:
#test_word = ['attraction merdique']
test_word = ['jurong point crazy available bugis great world la buffet cine amore wat']
test_word = tokenizer(test_word, return_tensors="np", padding=True)

In [ ]:
# test_word_tokenized[0]
# new_text_padded = tf.keras.preprocessing.sequence.pad_sequences(test_word_tokenized, padding="post", maxlen=reviews_pad.shape[1])

In [71]:
# Make a prediction
prediction = model.predict(test_word)

print("Prediction:", prediction)

1/1 [==============================] - 3s 3s/step
Prediction: TFSequenceClassifierOutput(loss=None, logits=array([[ 0.8749457, -1.5998322]], dtype=float32), hidden_states=None, attentions=None)
